In [2]:
import torch
import os
import json
import zipfile
import gzip
import matplotlib.pyplot as plt
from PIL import Image
import torchvision 
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import wandb
import tqdm
import torch.nn.functional as F

In [16]:
import argparse
import os
import random
import shutil
import time
import warnings
from enum import Enum
#from zipfile import ZipFile

from zippedimagefolder import ZippedDatasetFolder
import zippedimagefolder

import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Subset
import pytorch_lightning as pl
#%load_ext tensorboard

In [4]:
from model import AutoEncoder, HookedModel, DictionnaryLearner, default_hookedmodel, default_ae

2023-11-16 19:31:36.918678: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: david-heurtel-depeiges. Use `wandb login --relogin` to force relogin


True

In [7]:
!bash dataload.sh

/tmp/imagenet ~/VisionMonoSemanticity
~/VisionMonoSemanticity


In [24]:
## Config

## Data
TRAIN_PATH = '/tmp/imagenet/train'
VAL_PATH = '/tmp/imagenet/valid'

## Data transforms
PATCH_SIZE = 'Full'

## Model 
D_HIDDEN = 1024
CHANNELS = 508
D_MULT = 1 ## If not 1, D_HIDDEN = CHANNELS * D_MULT
if D_MULT != 1:
    D_HIDDEN = CHANNELS * D_MULT
L1_COEFF = 3e-4


MODEL_NAME = 'inceptionv1_mixed4a_monosemantic_dummy'

## Training
BATCH_SIZE = 4096
OPTIM = 'Adam'


## Checkpointing
SAVE_PATH = '/mnt/home/dheurtel/ceph/02_checkpoints/monosemantic/'

## Logging

WANBD_PROJECT = 'monosemantic_dictionnary_learning'

os.makedirs(SAVE_PATH, exist_ok=True)

In [6]:
CONFIG = {
    'dataset': 'imagenet',
    'patch_size': PATCH_SIZE,
    'd_input' : CHANNELS,
    'd_hidden': D_HIDDEN,
    'l1_coeff': L1_COEFF,
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'optim': OPTIM,
    'save_path': SAVE_PATH,
    'wandb_project': WANBD_PROJECT, 
}

In [ ]:
dirnotempy = os.path.exists('/tmp/imagenet/train')
if dirnotempy:
    l = os.listdir('/tmp/imagenet/train')
    if len(l)==0:
        dirnotempy = False
if not dirnotempy:
    os.system('bash dataload.sh')
TRAIN_DIR = '/tmp/imagenet/train/'
TEST_DIR = '/tmp/imagenet/train/'

traindir = os.path.join('/tmp/imagenet/', 'train')
valdir = os.path.join('/tmp/imagenet/', 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
def unormalize(batch):
    ## batch of shape (batch_size, 3, 224, 224)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

    return batch * std + mean

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

val_dataset = datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))    

In [7]:
## Create a dummy torch dataset that yields random images of size 3 x 224 x 224

class DummyDataset(Dataset):
    def __init__(self, size):
        self.size = size
    def __len__(self):
        return self.size
    def __getitem__(self, idx):
        return torch.rand(3, 224, 224), torch.zeros(1)
    
dummy_dataset = DummyDataset(1000)

dummy_dataloader = DataLoader(dummy_dataset, batch_size=128, shuffle=True, num_workers=4)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
dict_learner = DictionnaryLearner(d_hidden = 10 * 508, d_input = 508, l1_coeff = 1e-2)

In [10]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=8)

NameError: name 'train_dataset' is not defined

In [26]:
logger_type = 'tensorboard'

In [27]:
if logger_type.lower()=='wandb':
    logger = pl.loggers.WandbLogger(project=WANBD_PROJECT, config=CONFIG)
    logger.watch(dict_learner)
    logger.log_hyperparams(CONFIG)
    default_root_dir = None
elif logger_type.lower()=='tensorboard':
    logger = pl.loggers.TensorBoardLogger(SAVE_PATH, name=MODEL_NAME)
    default_root_dir = None
else:
    logger = None
    default_root_dir = SAVE_PATH

logger = pl.loggers.TensorBoardLogger(SAVE_PATH, name=MODEL_NAME)

trainer = pl.Trainer(gpus=1, max_epochs=1, progress_bar_refresh_rate=20,default_root_dir=default_root_dir, logger=logger)
trainer.fit(dict_learner, train_dataloaders=dummy_dataloader, val_dataloaders=dummy_dataloader)

/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type        | Params
--------------------------------------------
0 | autoencoder | AutoEncoder | 5.2 M 
1 | hookedmodel | HookedModel | 7.1 M 
--------------------------------------------
5.2 M     Trainable params
7.1 M     Non-trainable params
12.2 M    Total params
48.933    Total esti

Validation sanity check: 0it [00:00, ?it/s]

/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/mnt/sw/nix/store/pvz51gzx2lyqpabf08xhd1y2kvw5rfdy-python-3.9.16-view/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:407: UserWarning: The number of training samples (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Visualize the learned dictionary

In [65]:
## Get the dictionary of classes for ImageNet

CLASSES_FILE = './classes/imagenet1000_clsidx_to_labels.txt'

with open(CLASSES_FILE) as f:
    idx2classes = eval(f.read())

classes2idx = {v: k for k, v in idx2classes.items()}

In [131]:
## Build a tree for ImageNet classes using WordNet

def get_wordnet_tree():
    wnet_tree = {}
    count = 0
    for synset in wn.all_synsets():
        if synset.pos() != 'n':
            continue
        if not synset.instance_hypernyms():
            wnet_tree[synset.name()] = {}
        else:
            wnet_tree[synset.name()] = {hypo.name(): {} for hypo in synset.instance_hypernyms()}
    return wnet_tree

wnet_tree = get_wordnet_tree()

In [132]:
## Plot wnet_tree

def plot_tree(tree, depth=0):
    for k, v in tree.items():
        print('    ' * depth + k)
        plot_tree(v, depth+1)

In [140]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [133]:
plot_tree(wnet_tree)

entity.n.01
physical_entity.n.01
abstraction.n.06
thing.n.12
object.n.01
whole.n.02
congener.n.03
living_thing.n.01
organism.n.01
benthos.n.02
dwarf.n.03
heterotroph.n.01
parent.n.02
life.n.10
biont.n.01
cell.n.02
causal_agent.n.01
person.n.01
animal.n.01
plant.n.02
native.n.03
natural_object.n.01
substance.n.01
substance.n.07
matter.n.03
food.n.01
nutrient.n.02
artifact.n.01
article.n.02
psychological_feature.n.01
cognition.n.01
motivation.n.01
attribute.n.02
state.n.02
feeling.n.01
location.n.01
shape.n.02
time.n.05
space.n.01
absolute_space.n.01
phase_space.n.01
event.n.01
process.n.06
act.n.02
group.n.01
relation.n.01
possession.n.02
social_relation.n.01
communication.n.02
measure.n.02
phenomenon.n.01
thing.n.02
kindness.n.03
abdominoplasty.n.01
abort.n.01
accomplishment.n.01
agon.n.01
alienation.n.04
application.n.07
beachhead.n.02
cakewalk.n.02
feat.n.01
masterpiece.n.02
masterstroke.n.01
credit.n.04
action.n.01
res_gestae.n.02
course.n.04
blind_alley.n.02
collision_course.n.02
i

In [143]:
import nltk
from nltk.util import acyclic_depth_first as acyclic_tree
wn=nltk.corpus.wordnet
from pprint import pprint
acyclic_tree(wn.synset('dog.n.01'), lambda s:s.hypernyms(),cut_mark='...')

/tmp/ipykernel_710893/3294271493.py:5: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth -3
  acyclic_tree(wn.synset('dog.n.01'), lambda s:s.hypernyms(),cut_mark='...')


[Synset('dog.n.01'),
 [Synset('canine.n.02'),
  [Synset('carnivore.n.01'),
   [Synset('placental.n.01'),
    [Synset('mammal.n.01'),
     [Synset('vertebrate.n.01'),
      [Synset('chordate.n.01'),
       [Synset('animal.n.01'),
        [Synset('organism.n.01'),
         [Synset('living_thing.n.01'),
          [Synset('whole.n.02'),
           [Synset('object.n.01'),
            [Synset('physical_entity.n.01'),
             [Synset('entity.n.01')]]]]]]]]]]]]],
 [Synset('domestic_animal.n.01'), "Cycle(Synset('animal.n.01'),-3,...)"]]

In [149]:
wn.synsets('courgette')

[Synset('zucchini.n.01'), Synset('zucchini.n.02')]

In [151]:
import networkx as nx

In [152]:
## Build the whole graph of ImageNet classes with all relations (hypernyms, hyponyms, meronyms, holonyms, etc.)
## Each edge is 'weighted' by a list of relations between the two nodes
## The graph is directed

def get_wnet_graph():
    wnet_graph = nx.DiGraph()
    for synset in wn.all_synsets():
        if synset.pos() != 'n':
            continue
        wnet_graph.add_node(synset.name())
        for hypernym in synset.hypernyms():
            wnet_graph.add_edge(synset.name(), hypernym.name(), relations=['hypernym'])
        for hyponym in synset.hyponyms():
            wnet_graph.add_edge(synset.name(), hyponym.name(), relations=['hyponym'])
        for meronym in synset.part_meronyms():
            wnet_graph.add_edge(synset.name(), meronym.name(), relations=['part_meronym'])
        for holonym in synset.part_holonyms():
            wnet_graph.add_edge(synset.name(), holonym.name(), relations=['part_holonym'])
        for meronym in synset.substance_meronyms():
            wnet_graph.add_edge(synset.name(), meronym.name(), relations=['substance_meronym'])
        for holonym in synset.substance_holonyms():
            wnet_graph.add_edge(synset.name(), holonym.name(), relations=['substance_holonym'])
        for meronym in synset.member_meronyms():
            wnet_graph.add_edge(synset.name(), meronym.name(), relations=['member_meronym'])
        for holonym in synset.member_holonyms():
            wnet_graph.add_edge(synset.name(), holonym.name(), relations=['member_holonym'])
    return wnet_graph


wnet_graph = get_wnet_graph()

In [154]:
wnet_graph['window.n.01']

AtlasView({'framework.n.03': {'relations': ['hypernym']}, 'bay_window.n.02': {'relations': ['hyponym']}, 'casement_window.n.01': {'relations': ['hyponym']}, 'clerestory.n.01': {'relations': ['hyponym']}, 'display_window.n.01': {'relations': ['hyponym']}, 'dormer.n.01': {'relations': ['hyponym']}, 'dormer_window.n.01': {'relations': ['hyponym']}, 'double-hung_window.n.01': {'relations': ['hyponym']}, 'double_glazing.n.01': {'relations': ['hyponym']}, 'fanlight.n.03': {'relations': ['hyponym']}, 'lancet_window.n.01': {'relations': ['hyponym']}, 'louvered_window.n.01': {'relations': ['hyponym']}, 'oeil_de_boeuf.n.01': {'relations': ['hyponym']}, 'picture_window.n.01': {'relations': ['hyponym']}, 'pivoting_window.n.01': {'relations': ['hyponym']}, 'porthole.n.01': {'relations': ['hyponym']}, 'rose_window.n.01': {'relations': ['hyponym']}, 'sash_window.n.01': {'relations': ['hyponym']}, 'skylight.n.01': {'relations': ['hyponym']}, 'sliding_window.n.01': {'relations': ['hyponym']}, 'stained-

In [156]:
## Is there an edge between window and building if true, display the relations between the two nodes

wnet_graph.has_edge('window.n.01', 'building.n.01')

wnet_graph['window.n.01']['building.n.01']

{'relations': ['part_holonym']}

In [158]:
import json

# Load the ImageNet class index mapping
with open('./classes/imagenet_class_index.json', 'r') as f:
    class_index = json.load(f)

# Print the first few entries in the mapping
for idx, (class_id, (wnid, label)) in enumerate(class_index.items()):
    print(f"{idx + 1}: Class ID {class_id}, WordNet ID {wnid}, Label {label}")
    if idx == 4:  # Print the first 5 entries for illustration
        break


1: Class ID 0, WordNet ID n01440764, Label tench
2: Class ID 1, WordNet ID n01443537, Label goldfish
3: Class ID 2, WordNet ID n01484850, Label great_white_shark
4: Class ID 3, WordNet ID n01491361, Label tiger_shark
5: Class ID 4, WordNet ID n01494475, Label hammerhead
